In [1]:
# %pdb
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import pathlib as pl
import PIL
import pandas as pd

In [2]:
TEM_root = pl.Path("../Figures/TEM/Junwei's Images")

ag_path = TEM_root / pl.Path('As Grown/Plain/')
fc_wide_path = TEM_root / pl.Path('Field Cooled/EELS_wide')
fc_close_path = TEM_root / pl.Path('Field Cooled/EELS_close')
fcvc_wide_path = TEM_root / pl.Path('Field Cooled Voltage Conditioned/EELS_wide')
fcvc_close_path = TEM_root / pl.Path('Field Cooled Voltage Conditioned/EELS_close')

ag_names = [name for name in ag_path.iterdir() if 'EELS_' in name.name]
fc_wide_names = [name for name in fc_wide_path.iterdir() if 'EELS_' in name.name]
fc_close_names = [name for name in fc_close_path.iterdir() if 'EELS_' in name.name]
fcvc_wide_names = [name for name in fcvc_wide_path.iterdir() if 'EELS_' in name.name]
fcvc_close_names = [name for name in fcvc_close_path.iterdir() if 'EELS_' in name.name]

ag_image = ag_path / 'EELS.png'
fc_wide_image = fc_wide_path / 'EELS.png'
fc_close_image = fc_close_path / 'EELS.png'
fcvc_wide_image = fcvc_wide_path / 'EELS.png'
fcvc_close_image = fcvc_close_path / 'EELS.png'

images_list = [ag_image, fc_wide_image, fc_close_image, fcvc_wide_image, fcvc_close_image]
eels_names_list = [ag_names, fc_wide_names, fc_close_names, fcvc_wide_names, fcvc_close_names]
labels_list = ["As Grown", "Field Cooled", "Field Cooled", "Field Cooled+Voltage Conditioned", "Field Cooled+Voltage Conditioned"]

keys = ['AG', 'FC_wide', 'FC_close', 'FCVC_wide', 'FCVC_close']
image_sizes = {'AG': 334, 'FC_wide': 329, 'FC_close': 6.53, 'FCVC_wide': 276, 'FCVC_close': 20.4}


element_linestyles = {'Si': '-', 'Co': '-', 'Ni': '-', 'Pt': '-', 'Gd': '-', 'O': '-', 'Al': '-'}
element_colors = {'Si': 'black','Co': 'midnightblue','Ni': 'darkgreen','Pt': 'grey','Gd': 'darkgoldenrod','O': 'sienna','Al': 'red'}


In [3]:
def load_image(path):
    return np.array(PIL.Image.open(path).convert('L'))

def integrate_y(arr, normalize=True):
    integral = np.sum(arr, axis=0)
    
    if normalize:
        sorted_integral = np.sort(integral)
        median_of_top_bins = np.median(sorted_integral[-50:])
        median_of_bottom_bins = np.median(sorted_integral[:50])
        return (integral - median_of_bottom_bins)/(median_of_top_bins-median_of_bottom_bins) # Normalize the integral
    else:
        return integral

def get_element(path):
    return path.stem.split('_')[-1].capitalize()

def generate_data_structure():
    data = dict()
    for key, image, element_paths in zip(keys, images_list, eels_names_list):
        data[key] = pd.DataFrame()
        for element_path in element_paths:
            element = get_element(element_path)
            data[key][element] = integrate_y(load_image(element_path), normalize=False)
#         data[key]['image'] = image
    return data

def write_pandas_data(data, output_dir):
    for key, df in data.items():
        with open('{}/{}_{}'.format(pl.Path(output_dir).as_posix(), key, 'EELS_table.csv'), 'w') as f:
            df.to_csv(f, index=False)

In [4]:
# data = generate_data_structure()
# write_pandas_data(data, '/home/pdmurray/Desktop/peyton/Projects/NiCoO_ElectricalEB/Workspace/')

In [5]:
data = generate_data_structure()
data.keys()

dict_keys(['AG', 'FC_wide', 'FC_close', 'FCVC_wide', 'FCVC_close'])